# Demo of a Log-normal mock

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nbodykit 
import nbodykit.lab as nb
import numpy as np
from nbodykit.cosmology import Planck15
from nbodykit import setup_logging
setup_logging() # turn on logging to screen
nbodykit.__version__

In [ ]:
BIAS     = 1.5
BOX      = 5274      # Mpc/h
NMESH    = 256
redshift = 0.85     
NBAR     = 1.947e-4 # h3/Mpc3
SEED     = 12345
cosmo    = nb.cosmology.Planck15
Plin     = nb.cosmology.LinearPower(cosmo, redshift, transfer='CLASS')

# generate the catalog
cat      = nb.LogNormalCatalog(Plin=Plin, nbar=NBAR, BoxSize=BOX,
                               Nmesh=NMESH, bias=BIAS, seed=SEED)

In [ ]:
cat.attrs

In [ ]:
cat.columns

In [ ]:
cat['Position'].shape

In [ ]:
pos = cat['Position'].compute()

In [ ]:
plt.hist2d(pos[:,0], pos[:,1], bins=120);plt.show()

## Compute the Power Spectrum

In [ ]:
mesh = cat.to_mesh(compensated=True, resampler='cic', 
                   position='Position', BoxSize=BOX, Nmesh=256)
rpol = nb.FFTPower(mesh, mode='1d', poles=[0])

In [ ]:
rpol.attrs

In [ ]:
krange = np.logspace(-3, -1)
plt.plot(krange, BIAS**2*Plin(krange), 'k-')
for i in [0]:
    plt.loglog(rpol.poles['k'], rpol.poles['power_'+str(i)].real -rpol.attrs['shotnoise'],
               label=r'$\ell$ = %d'%i)
plt.legend()